In [1]:
# Importing necessary libraries
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report
import csv



# Observation:
Since the dataset having multiple errored lines for this project I have removed the errored lines and still secured 1231864 records

In [ ]:
def custom_error_handler(exception):
    # Implement your custom error handling logic here
    print(f"Error during parsing: {exception}")
    print(f"Problematic row: {exception.line}")

try:
    # Try to load the CSV file with custom headers
    df_new = pd.read_csv('/content/new_twitter.csv', error_bad_lines=False, sep=',', quoting=csv.QUOTE_NONE, encoding='latin-1', names=['target', 'ids', 'date', 'flag', 'user', 'text'])

except pd.errors.ParserError as e:
    print(f"Error loading dataset: {e}")

    # Retry loading without custom headers for the first row
    try:
        df_new = pd.read_csv('/content/new_twitter.csv', error_bad_lines=False, sep=',', quoting=csv.QUOTE_NONE, encoding='latin-1')
    except pd.errors.ParserError as e:
        print(f"Error loading dataset without custom headers: {e}")

In [3]:
df_new.head()
df_new.shape

(1231864, 6)

In [4]:
df = df_new[1:]
df.head()

,target,ids,date,flag,user,text
1,"""0""","""1467810672""","""Mon Apr 06 22:19:49 PDT 2009""","""NO_QUERY""","""scotthamilton""","""is upset that he can't update his Facebook by..."
2,"""0""","""1467810917""","""Mon Apr 06 22:19:53 PDT 2009""","""NO_QUERY""","""mattycus""","""@Kenichan I dived many times for the ball. Ma..."
3,"""0""","""1467811184""","""Mon Apr 06 22:19:57 PDT 2009""","""NO_QUERY""","""ElleCTF""","""my whole body feels itchy and like its on fire """
4,"""0""","""1467811372""","""Mon Apr 06 22:20:00 PDT 2009""","""NO_QUERY""","""joy_wolf""","""@Kwesidei not the whole crew """
5,"""0""","""1467811592""","""Mon Apr 06 22:20:03 PDT 2009""","""NO_QUERY""","""mybirch""","""Need a hug """


In [5]:
df.shape

(1231863, 6)

# Preprocessing


In [6]:
#  NLTK library
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
import re
import nltk

nltk.download('stopwords')



[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [7]:
df.drop_duplicates(inplace=True)
df.dropna(inplace=True)

<ipython-input-7-2960a2f9b025>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.drop_duplicates(inplace=True)
<ipython-input-7-2960a2f9b025>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.dropna(inplace=True)


df.shape

In [23]:
df.target.value_counts()

"0"    626730
"4"    605133
Name: target, dtype: int64

Due to my system compatibility im unable to train using the entire dataset
In this data the target colun=mn have 2 kinds of data 0 and 4
hence i reduced the data to 60000 where 30K from target =0 and remaining from target =4

In [9]:
df_0 = df.iloc[:30000]
df_0

,target,ids,date,flag,user,text
1,"""0""","""1467810672""","""Mon Apr 06 22:19:49 PDT 2009""","""NO_QUERY""","""scotthamilton""","""is upset that he can't update his Facebook by..."
2,"""0""","""1467810917""","""Mon Apr 06 22:19:53 PDT 2009""","""NO_QUERY""","""mattycus""","""@Kenichan I dived many times for the ball. Ma..."
3,"""0""","""1467811184""","""Mon Apr 06 22:19:57 PDT 2009""","""NO_QUERY""","""ElleCTF""","""my whole body feels itchy and like its on fire """
4,"""0""","""1467811372""","""Mon Apr 06 22:20:00 PDT 2009""","""NO_QUERY""","""joy_wolf""","""@Kwesidei not the whole crew """
5,"""0""","""1467811592""","""Mon Apr 06 22:20:03 PDT 2009""","""NO_QUERY""","""mybirch""","""Need a hug """
...,...,...,...,...,...,...
29996,"""0""","""1573423283""","""Tue Apr 21 01:06:01 PDT 2009""","""NO_QUERY""","""polg""","""poo Singstar trophies are not retroactive mi..."
29997,"""0""","""1573423828""","""Tue Apr 21 01:06:10 PDT 2009""","""NO_QUERY""","""InYourEyes2410""","""@ashsimpsonwentz http://twitpic.com/3pdzd - t..."
29998,"""0""","""1573424114""","""Tue Apr 21 01:06:15 PDT 2009""","""NO_QUERY""","""YoungTrends""","""I'm SO sad my P&amp;S died today """
29999,"""0""","""1573424418""","""Tue Apr 21 01:06:20 PDT 2009""","""NO_QUERY""","""flumpy101""","""so much to do today and so little time in whi..."


In [10]:
df_4 = df.iloc[1200000:]
df_4

,target,ids,date,flag,user,text
1200001,"""4""","""2185623354""","""Mon Jun 15 17:39:37 PDT 2009""","""NO_QUERY""","""christy71""","""I shoud become a singer someday in the near f..."
1200002,"""4""","""2185623609""","""Mon Jun 15 17:39:38 PDT 2009""","""NO_QUERY""","""jordans1grl""","""@Dori23 DOH!!! We missed eachother again!! LU..."
1200003,"""4""","""2185623730""","""Mon Jun 15 17:39:39 PDT 2009""","""NO_QUERY""","""alexxcherin""","""bring on summer """
1200004,"""4""","""2185623763""","""Mon Jun 15 17:39:39 PDT 2009""","""NO_QUERY""","""esmithh""","""UPDATE: Erika is glad that Vinny doesn't thin..."
1200005,"""4""","""2185623836""","""Mon Jun 15 17:39:39 PDT 2009""","""NO_QUERY""","""JeanneBehr""","""@Sirjohn_writer The &quot;I love this song &a..."
...,...,...,...,...,...,...
1231859,"""4""","""2193601966""","""Tue Jun 16 08:40:49 PDT 2009""","""NO_QUERY""","""AmandaMarie1028""","""Just woke up. Having no school is the best fe..."
1231860,"""4""","""2193601969""","""Tue Jun 16 08:40:49 PDT 2009""","""NO_QUERY""","""TheWDBoards""","""TheWDB.com - Very cool to hear old Walt inter..."
1231861,"""4""","""2193601991""","""Tue Jun 16 08:40:49 PDT 2009""","""NO_QUERY""","""bpbabe""","""Are you ready for your MoJo Makeover? Ask me ..."
1231862,"""4""","""2193602064""","""Tue Jun 16 08:40:49 PDT 2009""","""NO_QUERY""","""tinydiamondz""","""Happy 38th Birthday to my boo of alll time!!!..."


# Concatinated those 2 new target dataframe

In [11]:
df_new = pd.concat([df_0,df_4], ignore_index=True)
df_new.shape

(61863, 6)

Preprocessing the 'text' column by Removing stopwords, non-alphabetic characters, usernames

In [12]:

# Function to preprocess the text
def preprocess_text(text):
    text = re.sub('@[^\s]+', '', text)  # Remove usernames
    text = re.sub('[^a-zA-Z]', ' ', text)  # Remove non-alphabetic characters
    text = text.lower()  # Convert to lowercase
    text = ' '.join([word for word in text.split() if word not in set(stopwords.words('english'))])  # Remove stopwords

    return text

df_new['text'] = df_new['text'].apply(preprocess_text)


df_new.text

0        upset update facebook texting might cry result...
1        dived many times ball managed save rest go bounds
2                         whole body feels itchy like fire
3                                               whole crew
4                                                 need hug
                               ...                        
61858                        woke school best feeling ever
61859    thewdb com cool hear old walt interviews http ...
61860                      ready mojo makeover ask details
61861    happy th birthday boo alll time tupac amaru sh...
61862                                 happy charitytuesday
Name: text, Length: 61863, dtype: object

In [13]:
# converting my target column to integers
df_new['target'] = df_new['target'].astype(str).str.replace('"', '').astype(int)


# Train test split

In [14]:
# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(df_new['text'], df_new['target'], test_size=0.2, random_state=42)

# Feature extraction using CountVectorizer


In [15]:
vectorizer = CountVectorizer()
X_train_vectorized = vectorizer.fit_transform(X_train)
X_test_vectorized = vectorizer.transform(X_test)





/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


# Model Building

In [ ]:
# Model 1: Multinomial Naive Bayes
nb_classifier = MultinomialNB()
nb_classifier.fit(X_train_vectorized, y_train)
nb_predictions = nb_classifier.predict(X_test_vectorized)

# Model 2: Logistic Regression
lr_classifier = LogisticRegression(max_iter=50)
lr_classifier.fit(X_train_vectorized, y_train)
lr_predictions = lr_classifier.predict(X_test_vectorized)

In [16]:
# Model 3: Random Forest
rf_classifier = RandomForestClassifier(n_estimators=10, random_state=42)
rf_classifier.fit(X_train_vectorized, y_train)
rf_predictions = rf_classifier.predict(X_test_vectorized)

In [17]:
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import GradientBoostingClassifier

# Model 4: Support Vector Machine (SVM)
svm_classifier = SVC() #kernel='linear')
svm_classifier.fit(X_train_vectorized, y_train)
svm_predictions = svm_classifier.predict(X_test_vectorized)

# Model 5: k-Nearest Neighbors (KNN)
knn_classifier = KNeighborsClassifier(n_neighbors=3)
knn_classifier.fit(X_train_vectorized, y_train)
knn_predictions = knn_classifier.predict(X_test_vectorized)

# Model 6: Gradient Boosting
gb_classifier = GradientBoostingClassifier(n_estimators=10, learning_rate=1.0, max_depth=1, random_state=42)
gb_classifier.fit(X_train_vectorized, y_train)
gb_predictions = gb_classifier.predict(X_test_vectorized)

# Evaluate the models


In [18]:
def evaluate_model(predictions, model_name):
    accuracy = accuracy_score(y_test, predictions)
    report = classification_report(y_test, predictions)
    print(f"{model_name} Model:")
    print(f"Accuracy: {accuracy:.2f}")
    print("Classification Report:\n", report)




Multinomial Naive Bayes Model:
Accuracy: 0.77
Classification Report:
               precision    recall  f1-score   support

           0       0.75      0.77      0.76      5911
           4       0.79      0.76      0.77      6462

    accuracy                           0.77     12373
   macro avg       0.77      0.77      0.77     12373
weighted avg       0.77      0.77      0.77     12373



In [ ]:

# Evaluate Model 1: Multinomial Naive Bayes
evaluate_model(nb_predictions, "Multinomial Naive Bayes")

In [19]:
# Evaluate Model 2: Logistic Regression
evaluate_model(lr_predictions, "Logistic Regression")

Logistic Regression Model:
Accuracy: 0.77
Classification Report:
               precision    recall  f1-score   support

           0       0.77      0.74      0.75      5911
           4       0.77      0.80      0.78      6462

    accuracy                           0.77     12373
   macro avg       0.77      0.77      0.77     12373
weighted avg       0.77      0.77      0.77     12373



In [20]:
# Evaluate Model 3: Random Forest
evaluate_model(rf_predictions, "Random Forest")

Random Forest Model:
Accuracy: 0.73
Classification Report:
               precision    recall  f1-score   support

           0       0.71      0.74      0.72      5911
           4       0.75      0.72      0.74      6462

    accuracy                           0.73     12373
   macro avg       0.73      0.73      0.73     12373
weighted avg       0.73      0.73      0.73     12373



In [21]:
# Evaluate Model 4: Support Vector Machine (SVM)
evaluate_model(svm_predictions, "Support Vector Machine (SVM)")
print('\n')




Support Vector Machine (SVM) Model:
Accuracy: 0.77
Classification Report:
               precision    recall  f1-score   support

           0       0.79      0.72      0.75      5911
           4       0.76      0.82      0.79      6462

    accuracy                           0.77     12373
   macro avg       0.77      0.77      0.77     12373
weighted avg       0.77      0.77      0.77     12373



k-Nearest Neighbors (KNN) Model:
Accuracy: 0.66
Classification Report:
               precision    recall  f1-score   support

           0       0.64      0.66      0.65      5911
           4       0.68      0.67      0.67      6462

    accuracy                           0.66     12373
   macro avg       0.66      0.66      0.66     12373
weighted avg       0.66      0.66      0.66     12373



Gradient Boosting Model:
Accuracy: 0.58
Classification Report:
               precision    recall  f1-score   support

           0       0.81      0.16      0.27      5911
           4       0.5

In [ ]:
# Evaluate Model 5: k-Nearest Neighbors (KNN)
evaluate_model(knn_predictions, "k-Nearest Neighbors (KNN)")
print('\n')



In [ ]:
# Evaluate Model 6: Gradient Boosting
evaluate_model(gb_predictions, "Gradient Boosting")

# Observation
Since Multinomial Naive Bayes, Logisitic Regression, SVM produces same accuracy considering F1 Score SVM Classifier performs the best

Need to perform Hyperparametric tuning to get better results